# RQ1 - Eyetracking

## Import Libraries

In [1]:
import sys
sys.path.append('../data/24/dataEvaluation/')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import utils.GenSnippetsLib as gsl
import math
import json


## Import Fixation Data

In [2]:
# function to convert a string seperated by whitespace characters back to python list
def string_to_list_string(data):
    data = data.replace(' ', ',')
    data = data.replace('\n', ',')
    data = ','.join([element for element in data.split(",") if len(element) > 0])
    if data[1] == ",":
        data = "[" + data[2:]
    return data

# read in the fixation data
df_fixation = pd.read_csv('../data/24/dataEvaluation/data/filteredData/fixation_stats.csv', sep=";")
# kick out outliers
df_fixation = df_fixation[df_fixation["IsOutlier"] == False]
df_fixation = df_fixation.drop(columns=["IsOutlier", "Behavioral"])

# transform fixation strings to lists
df_fixation["Fixation_startT"] = df_fixation["Fixation_startT"].apply(string_to_list_string)
df_fixation["Fixation_endT"] = df_fixation["Fixation_endT"].apply(string_to_list_string)
df_fixation["Fixation_x"] = df_fixation["Fixation_x"].apply(string_to_list_string)
df_fixation["Fixation_y"] = df_fixation["Fixation_y"].apply(string_to_list_string)
df_fixation["Fixation_x_range"] = df_fixation["Fixation_x_range"].apply(string_to_list_string)
df_fixation["Fixation_y_range"] = df_fixation["Fixation_y_range"].apply(string_to_list_string)

# Token Based Metrics

### Read in the Generator for Token Based Metrics to get The BoundingBoxes and Indices of each Token per Algorithm

In [3]:
# Get Token based AOIs
snippets = df_fixation["Algorithm"].unique()
df_token_aois = pd.DataFrame(columns=["Algorithm", "Token", "TokenIdx", "BoundingBox"])
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}_ast.json"
    image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/")
    height, width = image.size
    width_offset = int(1920 * 0.5) - int(height / 2)
    height_offset = int(1080 * 0.5) - int(width / 2)
    aoi_clustered = []
    current_left = None
    current_top = None
    current_right = None
    current_bottom = None
    current_aoi = None
    color = None
    for letter in aoi_list:
        if len(letter["AOI"]) == 1 or letter["letter"] == '\n':
            if current_aoi is not None:
                aoi_clustered.append(
                    (len(aoi_clustered), current_aoi, current_left, current_top, current_right, current_bottom, color))
            current_aoi = None
            color = None
            current_left = None
            current_top = None
            current_right = None
            current_bottom = None
            continue
        if current_aoi is None:
            current_aoi = letter["AOI"][1]
            color = letter["color"]
            current_left = letter["BoundingBox"][0]
            current_top = letter["BoundingBox"][1]
            current_right = letter["BoundingBox"][2]
            current_bottom = letter["BoundingBox"][3]
        elif current_aoi == letter["AOI"][1]:
            current_left = min(current_left, letter["BoundingBox"][0])
            current_top = min(current_top, letter["BoundingBox"][1])
            current_right = max(current_right, letter["BoundingBox"][2])
            current_bottom = max(current_bottom, letter["BoundingBox"][3])
        else:
            aoi_clustered.append(
                (len(aoi_clustered), current_aoi, current_left, current_top, current_right, current_bottom, color))
            current_aoi = letter["AOI"][1]
            color = letter["color"]
            current_left = letter["BoundingBox"][0]
            current_top = letter["BoundingBox"][1]
            current_right = letter["BoundingBox"][2]
            current_bottom = letter["BoundingBox"][3]

    for token in aoi_clustered:
        df_token_aois.loc[len(df_token_aois)] = [snippet, token[1], token[0],
                                                 (token[2] + width_offset,
                                                  token[3] + height_offset,
                                                  token[4] + width_offset,
                                                  token[5] + height_offset)]
df_token_aois

  0%|          | 0/32 [00:00<?, ?it/s]

AOI not closed - None
AOI not closed - None
AOI not closed - None


,Algorithm,Token,TokenIdx,BoundingBox
0,IsPrime,Modifier,0,"(798, 468, 846, 482)"
1,IsPrime,Modifier,1,"(854, 468, 902, 479)"
2,IsPrime,BasicType,2,"(910, 469, 966, 479)"
3,IsPrime,Identifier,3,"(974, 468, 1030, 479)"
4,IsPrime,Separator,4,"(1030, 467, 1038, 481)"
...,...,...,...,...
2488,Rectangle,Keyword,82,"(977, 677, 1009, 688)"
2489,Rectangle,Separator,83,"(1009, 685, 1017, 688)"
2490,Rectangle,Identifier,84,"(1017, 677, 1065, 691)"
2491,Rectangle,Separator,85,"(1065, 676, 1089, 691)"


### Check which Fixation of which Participant is in which Token

In [4]:
df_token_fixation_per_participant = pd.DataFrame([], columns=["Algorithm", "Participant", "FixationNumber",
                                                            "FixationDuration", "TokenIdx"])
participants = df_fixation["Participant"].unique()
print(participants)
for snippet in tqdm(snippets):
    df_token_per_algo = df_token_aois[df_token_aois["Algorithm"] == snippet]

    for participant in participants:
        df_fixation_participant = df_fixation[
            (df_fixation["Algorithm"] == snippet) & (df_fixation["Participant"] == participant)]
        if len(df_fixation_participant) == 0:
            continue
        start_times = eval(df_fixation_participant["Fixation_startT"].values[0])
        end_times = eval(df_fixation_participant["Fixation_endT"].values[0])
        x_coordinates = eval(df_fixation_participant["Fixation_x"].values[0])
        y_coordinates = eval(df_fixation_participant["Fixation_y"].values[0])
        x_range = eval(df_fixation_participant["Fixation_x_range"].values[0])
        y_range = eval(df_fixation_participant["Fixation_y_range"].values[0])
        idx_values = range(len(start_times))
        for (fix_idx, start, end, x, y, x_range, y_range) in zip(idx_values, start_times, end_times, x_coordinates, y_coordinates, x_range, y_range):
            low_x = int(float(x) - math.ceil(float(x_range)))
            low_y = int(float(y) - math.ceil(float(y_range)))
            high_x = int(float(x) + math.ceil(float(x_range)))
            high_y = int(float(y) + math.ceil(float(y_range)))
            possible_coordinates = [(x, y) for x in range(low_x, high_x + 1) for y in range(low_y, high_y + 1)]

            found = False
            for idx, row in df_token_per_algo.iterrows():
                token_idx = row["TokenIdx"]
                bounding_box = row["BoundingBox"]

                for possible_x, possible_y in possible_coordinates:
                    if bounding_box[0] <= possible_x <= bounding_box[2] and bounding_box[1] <= possible_y <=bounding_box[3]:
                        df_token_fixation_per_participant.loc[len(df_token_fixation_per_participant)] = [snippet, participant,fix_idx,end - start,token_idx]
                        found = True
                        break
                if found:
                    break

df_token_fixation_per_participant

[ 1  2  3  4  5  6  7 10 11 12 13 14 18 22 24 25 28 35 36 37 38 41 42 49
 50 55 58 59 60 61 62 63 66 67 68 70 71]


  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,Participant,FixationNumber,FixationDuration,TokenIdx
0,IsPrime,1,4,160.005,0
1,IsPrime,1,6,296.008,1
2,IsPrime,1,7,780.019,3
3,IsPrime,1,10,220.001,10
4,IsPrime,1,11,200.006,14
...,...,...,...,...,...
39283,Rectangle,70,71,352.006,66
39284,Rectangle,71,9,628.018,47
39285,Rectangle,71,16,216.006,43
39286,Rectangle,71,17,776.027,42


### Transform the Data to a Fixation/ Refixation split by Participant

In [5]:
df_token = df_token_aois.copy()
df_token = df_token.drop(["BoundingBox", "Token"], axis=1)
for participant in participants:
    df_token.loc[:, f"TokenFixation_P{participant}"] = [[] for _ in range(len(df_token))]
    df_token.loc[:, f"TokenReFixation_P{participant}"] = [[] for _ in range(len(df_token))]

prev_participant = df_token_fixation_per_participant["Participant"].iloc[0]
prev_token_idx = df_token_fixation_per_participant["TokenIdx"].iloc[0]
prev_algorithm = df_token_fixation_per_participant["Algorithm"].iloc[0]
fixations = []
re_fixation = False
for idx, row in tqdm(df_token_fixation_per_participant.iterrows(), total=len(df_token_fixation_per_participant)):
    participant = row["Participant"]
    token_idx = row["TokenIdx"]
    algorithm = row["Algorithm"]
    FixationDuration = row["FixationDuration"]

    # fixation switches
    if prev_participant != participant or prev_token_idx != token_idx:
        index = df_token[(df_token["TokenIdx"] == prev_token_idx) & (df_token["Algorithm"] == prev_algorithm)].index[0]
        if re_fixation:
            re_fixations = df_token.loc[index, f"TokenReFixation_P{prev_participant}"]
            re_fixations.extend(fixations.copy())
            df_token.at[index, f"TokenReFixation_P{prev_participant}"] = re_fixations.copy()
        else:
            df_token.at[index, f"TokenFixation_P{prev_participant}"] = fixations.copy()
        fixations = []
        # possible new fixation
        re_fixation = False

    sub_frame = df_token[(df_token["TokenIdx"] == token_idx) & (df_token["Algorithm"] == algorithm)]
    if len(sub_frame) == 0:
        raise Exception(f"No Token found for {token_idx} in Algorithm {algorithm}")
    len_of_fixation = len(sub_frame[f"TokenFixation_P{participant}"].iloc[0])
    if re_fixation == False and len_of_fixation > 0 and len(fixations) == 0:
        re_fixation = True

    fixations.append(FixationDuration)
    prev_participant = participant
    prev_token_idx = token_idx
    prev_algorithm = algorithm

df_token

  0%|          | 0/39288 [00:00<?, ?it/s]

,Algorithm,TokenIdx,TokenFixation_P1,TokenReFixation_P1,TokenFixation_P2,TokenReFixation_P2,TokenFixation_P3,TokenReFixation_P3,TokenFixation_P4,TokenReFixation_P4,...,TokenFixation_P66,TokenReFixation_P66,TokenFixation_P67,TokenReFixation_P67,TokenFixation_P68,TokenReFixation_P68,TokenFixation_P70,TokenReFixation_P70,TokenFixation_P71,TokenReFixation_P71
0,IsPrime,0,[160.00499999999988],[],[300.01099999999997],[160.0050000000001],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
1,IsPrime,1,[296.0079999999998],[],[392.01199999999994],"[364.0169999999998, 100.0039999999999]",[224.00700000000143],[],[],[],...,[],[],[620.0129999999999],"[1112.0370000000003, 688.0220000000008, 236.00...",[376.0119999999997],[],[],[],[],[]
2,IsPrime,2,[],[],[196.0150000000001],"[216.00599999999986, 180.00800000000027, 244.0...",[492.01800000000003],[],[444.0110000000004],[212.0059999999994],...,[],[],[1956.0699999999997],"[239.9960000000001, 280.0109999999986, 2440.08...",[708.0219999999999],"[240.00700000000143, 372.01300000000265, 3272....",[],[],[],[]
3,IsPrime,3,[780.0190000000002],[208.0079999999998],[1000.04],[],[],[],"[1104.039, 624.0219999999999]",[],...,"[160.00400000000081, 312.009]",[528.0260000000017],[744.028],"[904.031, 280.0159999999996, 500.0180000000000...",[324.0129999999999],[],[],[],[],[]
4,IsPrime,4,[],[],[],[],[],[],[],[],...,[],[],[216.0099999999984],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,Rectangle,82,[296.0120000000006],[380.0139999999992],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2489,Rectangle,83,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2490,Rectangle,84,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2491,Rectangle,85,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


In [6]:
# Dump df_token to csv at raw data folder
df_token.to_csv("../data/24/dataEvaluation/data/filteredData/token_fixations.csv", sep=";", index=False)

### Calculate the Token Based Eyetracking Metrics

In [7]:
# Melt the Dataframe to be thinner so that we have Algorithm TokenIdx and Participant and Keys per Fixation / Refixation
df_token_melted = pd.melt(df_token, id_vars=["Algorithm", "TokenIdx"], var_name="KindOfFixation", value_name="FixationDurations")

print(df_token_melted)

# Classify Participant and the Kind and Number of Fixations / Refixations
df_token_melted["Participant"] = df_token_melted["KindOfFixation"].apply(lambda x: int(x.split("_")[1][1:]))
df_token_melted["KindOfFixation"] = df_token_melted["KindOfFixation"].apply(lambda x: x.split("_")[0])
df_token_melted["KindOfFixation"] = df_token_melted["KindOfFixation"].apply(lambda x: "Fixation" if x == "TokenFixation" else "ReFixation")
df_token_melted["NumberOfFixations"] = df_token_melted["FixationDurations"].apply(lambda x: len(x))

# Get the number of Participants for further calculations
number_of_participants = len(participants)

# Calculate the First Fixation Duration per Participant per Token
df_token_melted["FirstFixationDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "FirstFixationDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) > 0 else None)

# Calculate the Single Fixation Duration per Participant per Token
df_token_melted["SingleFixationDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "SingleFixationDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) == 1 else None)

# Calculate the Gaze Duration per Participant per Token
df_token_melted["GazeDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "GazeDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: sum(x) if len(x) > 0 else None)

# Calculate the Total Time per Participant per Token
df_token_melted_total_time = df_token_melted.groupby(["Participant", "Algorithm", "TokenIdx"])\
    .agg({"FixationDurations": lambda x: sum(x.values.sum())})
df_token_melted_total_time = df_token_melted_total_time.rename(columns={"FixationDurations": "TotalTime"})

# Merge the Dataframes
df_token_melted = pd.merge(df_token_melted, df_token_melted_total_time, on=["Participant", "Algorithm", "TokenIdx"], how="left")

# Cast the Dataframes to the right datatype
df_token_melted["FirstFixationDuration"] = df_token_melted["FirstFixationDuration"].astype(float)
df_token_melted["SingleFixationDuration"] = df_token_melted["SingleFixationDuration"].astype(float)
df_token_melted["GazeDuration"] = df_token_melted["GazeDuration"].astype(float)
df_token_melted["TotalTime"] = df_token_melted["TotalTime"].astype(float).replace(0, np.nan)

# Read in the Skilllevel
df_skill = pd.read_csv(f"../data/24/dataEvaluation/data/filteredData/filtered_data.csv")
df_skill = df_skill[["Participant", "SkillScore"]]
df_skill = df_skill.drop_duplicates()

# Merge the Dataframes to combine metrics with the Skilllevel
df_metrics_skill = pd.merge(df_token_melted, df_skill, on=["Participant"], how="left")

# Helper Methods for the Metrics
def get_no_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 0]
    return df_fixations


def get_single_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[~df_fixations["TokenIdx"].isin(df_refixations["TokenIdx"].values)]
    # remove every entry from df fixations on where Number Of Fixations is not 1
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 1]
    return df_fixations

def get_multiple_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[(df_fixations["TokenIdx"].isin(df_refixations["TokenIdx"].values)) & (df_fixations["NumberOfFixations"] >= 1)]
    return df_fixations

def get_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on where no Fixation is found
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] >= 1]
    return df_fixations



# dataframe for number of fixations per participant
number_of_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])["NumberOfFixations"].count()
number_of_fixation_per_algorithm = number_of_fixation_per_algorithm.reset_index()

# dataframe for number of tokens per algorithm
number_of_tokens_per_algorithm = df_metrics_skill.groupby(["Algorithm"])["TokenIdx"].max()
number_of_tokens_per_algorithm = number_of_tokens_per_algorithm.reset_index()

# dataframe for number of tokens with no fixation per algorithm per participant
df_no_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_no_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with only one fixation per algorithm per participant
df_single_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_single_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with more than one fixation per algorithm per participant
df_multiple_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_multiple_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with more or equal than one fixation per algorithm per participant
df_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_fixations)\
    .drop(["Algorithm"], axis=1)

# Reformat the dataframes
no_fixation_per_algorithm = df_no_fixation_per_algorithm[["TokenIdx"]]
no_fixation_per_algorithm = no_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

single_fixation_per_algorithm = df_single_fixation_per_algorithm[["TokenIdx"]]
single_fixation_per_algorithm = single_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

multiple_fixation_per_algorithm = df_multiple_fixation_per_algorithm[["TokenIdx"]]
multiple_fixation_per_algorithm = multiple_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

fixations_per_algorithm = df_fixation_per_algorithm[["TokenIdx"]]
fixations_per_algorithm = fixations_per_algorithm.reset_index().drop(["level_2"], axis=1)

# Helper Method for Probability Metrics
def group_len_divided_by_number(current_df, counting_df):
    algorithm = current_df["Algorithm"].iloc[0]
    number_of_tokens = counting_df[counting_df["Algorithm"] == algorithm]["TokenIdx"].iloc[0]
    value = len(current_df) / number_of_tokens
    return len(current_df) / (number_of_tokens + 1)

# Calculate the Metrics per Participant
# Probability of no fixation
df_no_fixation_probability = no_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_no_fixation_probability = df_no_fixation_probability.reset_index()

# Probability of single fixation
df_single_fixation_probability = single_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_single_fixation_probability = df_single_fixation_probability.reset_index()

# Probability of multiple fixation
df_multiple_fixation_probability = multiple_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_multiple_fixation_probability = df_multiple_fixation_probability.reset_index()

# Probability of fixation
df_fixation_probability = fixations_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_fixation_probability = df_fixation_probability.reset_index()

# Calculate the means for the metrics per algorithm
df_no_fixation_probability = df_no_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_single_fixation_probability = df_single_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_multiple_fixation_probability = df_multiple_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_fixation_probability = df_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)

# Raw Durations Metrics
# Duration of first fixation
df_first_fixation = df_metrics_skill[~df_metrics_skill["FirstFixationDuration"].isnull()]
df_first_fixation = df_first_fixation.groupby(["Participant"])["FirstFixationDuration"].mean()

# Duration of single fixation
df_single_fixation = df_metrics_skill[~df_metrics_skill["SingleFixationDuration"].isnull()]
df_single_fixation = df_single_fixation.groupby(["Participant"])["SingleFixationDuration"].mean()

# Duration of gaze duration
df_gaze_duration = df_metrics_skill[~df_metrics_skill["GazeDuration"].isnull()]
df_gaze_duration = df_gaze_duration.groupby(["Participant"])["GazeDuration"].mean()

# Total Time
df_total_time = df_metrics_skill[~df_metrics_skill["TotalTime"].isnull()]
df_total_time = df_total_time.groupby(["Participant"])["TotalTime"].mean()

# Put every metric dataframe together into one
df_combined = pd.DataFrame({"FirstFixationDuration": df_first_fixation.values,
                            "SingleFixationDuration": df_single_fixation.values,
                            "GazeDuration": df_gaze_duration.values,
                            "TotalTime": df_total_time.values,
                            "TokenNoFixationProbability": df_no_fixation_probability.values.reshape(37, ),
                            "TokenSingleFixationProbability": df_single_fixation_probability.values.reshape(37, ),
                            "TokenMultipleFixationProbability": df_multiple_fixation_probability.values.reshape(37, ),
                            "TokenFixationProbability": df_fixation_probability.values.reshape(37, ),
                            "Skill": df_metrics_skill.groupby(["Participant"])["SkillScore"].mean().values})
# get spearman correlation for metrics and skill level
df_combined.corrwith(df_combined["Skill"])

        Algorithm  TokenIdx       KindOfFixation     FixationDurations
0         IsPrime         0     TokenFixation_P1  [160.00499999999988]
1         IsPrime         1     TokenFixation_P1   [296.0079999999998]
2         IsPrime         2     TokenFixation_P1                    []
3         IsPrime         3     TokenFixation_P1   [780.0190000000002]
4         IsPrime         4     TokenFixation_P1                    []
...           ...       ...                  ...                   ...
184477  Rectangle        82  TokenReFixation_P71                    []
184478  Rectangle        83  TokenReFixation_P71                    []
184479  Rectangle        84  TokenReFixation_P71                    []
184480  Rectangle        85  TokenReFixation_P71                    []
184481  Rectangle        86  TokenReFixation_P71                    []

[184482 rows x 4 columns]


FirstFixationDuration              -0.083683
SingleFixationDuration             -0.085188
GazeDuration                       -0.151251
TotalTime                          -0.441480
TokenNoFixationProbability          0.161758
TokenSingleFixationProbability     -0.390973
TokenMultipleFixationProbability   -0.561149
TokenFixationProbability           -0.534995
Skill                               1.000000
dtype: float64

In [8]:
df_combined

,FirstFixationDuration,SingleFixationDuration,GazeDuration,TotalTime,TokenNoFixationProbability,TokenSingleFixationProbability,TokenMultipleFixationProbability,TokenFixationProbability,Skill
0,349.140230,339.856561,391.967603,747.749891,0.682866,0.160839,0.144605,0.317134,0.331385
1,323.081535,323.384355,361.394677,683.916987,0.677890,0.176096,0.136825,0.322110,0.379187
2,403.691889,397.778311,445.915592,858.063653,0.817145,0.210210,0.173833,0.390090,0.311264
3,326.450693,322.436709,357.657468,654.965319,0.745821,0.142297,0.105922,0.254179,0.424727
4,430.852016,432.196450,490.078586,893.535275,0.658431,0.178554,0.152347,0.341569,0.313031
5,338.656379,339.861908,361.626130,710.636081,0.670818,0.184138,0.181422,0.376208,0.315932
6,311.408738,309.404445,322.278809,681.586404,0.761581,0.152338,0.121480,0.272479,0.420873
7,376.626817,378.579796,406.196297,622.436008,0.781150,0.138042,0.099739,0.233440,0.350392
8,318.820969,321.301841,334.888210,756.265388,0.774644,0.178879,0.189652,0.379547,0.178206
9,341.041769,340.424552,380.385434,805.490647,0.751547,0.157738,0.125850,0.294462,0.309233


In [9]:
# Save the Dataframe to a CSV
df_combined.to_csv("./original_24_metrics.csv", index=False)

# AOI based analysis

### Read in the Generator for AOI Based Metrics to get The BoundingBoxes and Indices of each Token per Algorithm

In [10]:
# Get Token based AOIs
snippets = df_fixation["Algorithm"].unique()
df_aois = pd.DataFrame(columns=["Algorithm", "AOI", "AOIIdx", "BoundingBox"])
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}.json"
    try:
        image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/")
    except:
        print(f"{snippet} failed")
        image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/", logging=True)
        break
    height, width = image.size
    width_offset = int(1920 * 0.5) - int(height / 2)
    height_offset = int(1080 * 0.5) - int(width / 2)
    aoi_clustered = []
    current_left = []
    current_top = []
    current_right = []
    current_bottom = []
    current_aoi = []
    color = []
    for letter in aoi_list:
        # Close AOI
        if letter['letter'] in " \t\n":
            continue
        if len(letter["AOI"]) == 1:
            for idx in range(len(current_aoi)):
                aoi_clustered.append((len(aoi_clustered), current_aoi[idx], current_left[idx], current_top[idx], current_right[idx], current_bottom[idx], color[idx]))

            current_aoi = []
            color = []
            current_left = []
            current_top = []
            current_right = []
            current_bottom = []
            continue
        # There is no AOI set
        if len(current_aoi) == 0:
            current_aoi = []
            color = []
            current_left = []
            current_top = []
            current_right = []
            current_bottom = []
            for idx in range(1, len(letter["AOI"])):
                current_aoi.append(letter["AOI"][idx])
                current_left.append(letter["BoundingBox"][0])
                current_top.append(letter["BoundingBox"][1])
                current_right.append(letter["BoundingBox"][2])
                current_bottom.append(letter["BoundingBox"][3])
                color.append(letter["color"])
            continue


        for idx in reversed(range(len(current_aoi))):
            if current_aoi[idx] in letter["AOI"]:
                current_left[idx] = min(current_left[idx], letter["BoundingBox"][0])
                current_top[idx] = min(current_top[idx], letter["BoundingBox"][1])
                current_right[idx] = max(current_right[idx], letter["BoundingBox"][2])
                current_bottom[idx] = max(current_bottom[idx], letter["BoundingBox"][3])
                # remove the AOI from the letter
                letter["AOI"].remove(current_aoi[idx])
            else:
                aoi_clustered.append((len(aoi_clustered), current_aoi[idx], current_left[idx], current_top[idx], current_right[idx], current_bottom[idx], color[idx]))
                del current_aoi[idx]
                del current_left[idx]
                del current_top[idx]
                del current_right[idx]
                del current_bottom[idx]
                del color[idx]

        for idx in range(1, len(letter["AOI"])):
            current_aoi.append(letter["AOI"][idx])
            current_left.append(letter["BoundingBox"][0])
            current_top.append(letter["BoundingBox"][1])
            current_right.append(letter["BoundingBox"][2])
            current_bottom.append(letter["BoundingBox"][3])
            color.append(letter["color"])

    for idx in range(len(current_aoi)):
        aoi_clustered.append((len(aoi_clustered), current_aoi[idx], current_left[idx], current_top[idx], current_right[idx], current_bottom[idx], color[idx]))

    for token in aoi_clustered:
        df_aois.loc[len(df_aois)] = [snippet, token[1], token[0],
                                                 (token[2] + width_offset,
                                                  token[3] + height_offset,
                                                  token[4] + width_offset,
                                                  token[5] + height_offset)]

df_aois

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,AOI,AOIIdx,BoundingBox
0,IsPrime,method_identifier,0,"(983, 444, 1060, 460)"
1,IsPrime,method_signature,1,"(741, 444, 1192, 464)"
2,IsPrime,method_argument_declaration,2,"(1060, 444, 1192, 463)"
3,IsPrime,for_head,3,"(785, 469, 1137, 489)"
4,IsPrime,arithmetic_expression,4,"(873, 494, 984, 510)"
...,...,...,...,...
626,Rectangle,method_call_identifier,14,"(1037, 769, 1103, 789)"
627,Rectangle,method_arguments,15,"(1103, 769, 1125, 788)"
628,Rectangle,method_call,16,"(982, 769, 1125, 789)"
629,Rectangle,arithmetic_expression,17,"(817, 769, 1125, 789)"


### Check which Fixation of which Participant is in which AOI

In [11]:
df_aoi_fixation_per_participant = pd.DataFrame([], columns=["Algorithm", "Participant", "FixationNumber",
                                                              "FixationDuration", "AOIIdx", "AOIName"])
participants = df_fixation["Participant"].unique()
for snippet in tqdm(snippets):
    df_aois_per_algo = df_aois[df_aois["Algorithm"] == snippet]

    for participant in tqdm(participants):

        df_fixation_participant = df_fixation[(df_fixation["Algorithm"] == snippet) & (df_fixation["Participant"] == participant)]
        if len(df_fixation_participant) == 0:
            continue
        start_times = eval(df_fixation_participant["Fixation_startT"].values[0])
        end_times = eval(df_fixation_participant["Fixation_endT"].values[0])
        x_coordinates = eval(df_fixation_participant["Fixation_x"].values[0])
        y_coordinates = eval(df_fixation_participant["Fixation_y"].values[0])
        x_range = eval(df_fixation_participant["Fixation_x_range"].values[0])
        y_range = eval(df_fixation_participant["Fixation_y_range"].values[0])
        idx_values = range(len(start_times))
        for (fix_idx, start, end, x, y, x_range, y_range) in zip(idx_values, start_times, end_times, x_coordinates, y_coordinates, x_range, y_range):
            low_x = int(float(x) - math.ceil(float(x_range)))
            low_y = int(float(y) - math.ceil(float(y_range)))
            high_x = int(float(x) + math.ceil(float(x_range)))
            high_y = int(float(y) + math.ceil(float(y_range)))
            possible_coordinates = [(x, y) for x in range(low_x, high_x + 1) for y in range(low_y, high_y + 1)]

            for idx, row in df_aois_per_algo.iterrows():
                aoi_idx = row["AOIIdx"]
                aoi_name = row["AOI"]
                bounding_box = row["BoundingBox"]

                for possible_x, possible_y in possible_coordinates:
                    if bounding_box[0] <= possible_x <= bounding_box[2] and bounding_box[1] <= possible_y <=bounding_box[3]:
                        df_aoi_fixation_per_participant.loc[len(df_aoi_fixation_per_participant)] = [snippet, participant, fix_idx ,end - start, aoi_idx, aoi_name]
                        break

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]


### Transform the Data to a Fixation/ Refixation split by Participant

In [12]:
df_aois_current = df_aois.copy()
df_aois_current = df_aois_current.drop(["BoundingBox"], axis=1)
for participant in participants:
    df_aois_current.loc[:, f"AOIFixation_P{participant}"] = df_aois_current.apply(lambda row: [], axis=1)
    df_aois_current.loc[:, f"AOIReFixation_P{participant}"] = df_aois_current.apply(lambda row: [], axis=1)


prev_participant = df_aoi_fixation_per_participant["Participant"].iloc[0]
prev_aoi_idx = df_aoi_fixation_per_participant["AOIIdx"].iloc[0]
prev_algorithm = df_aoi_fixation_per_participant["Algorithm"].iloc[0]
fixations = []
re_fixation = False
for idx, row in tqdm(df_aoi_fixation_per_participant.iterrows(), total=len(df_aoi_fixation_per_participant)):
    participant = row["Participant"]
    aoi_idx = row["AOIIdx"]
    algorithm = row["Algorithm"]
    FixationDuration = row["FixationDuration"]

    # fixation switches
    if prev_participant != participant or prev_aoi_idx != aoi_idx:
        index = df_aois_current[(df_aois_current["AOIIdx"] == prev_aoi_idx) & (df_aois_current["Algorithm"] == prev_algorithm)].index[0]
        if re_fixation:
            re_fixations = df_aois_current.loc[index, f"AOIReFixation_P{prev_participant}"]
            re_fixations.extend(fixations.copy())
            df_aois_current.at[index, f"AOIReFixation_P{prev_participant}"] = re_fixations.copy()
        else:
            df_aois_current.at[index, f"AOIFixation_P{prev_participant}"] = fixations.copy()
        fixations = []
        # possible new fixation
        re_fixation = False

    sub_frame = df_aois_current[(df_aois_current["AOIIdx"] == aoi_idx) & (df_aois_current["Algorithm"] == algorithm)]
    if len(sub_frame) == 0:
        raise Exception(f"No AOI found for {aoi_idx} in Algorithm {algorithm}")
    len_of_fixation = len(sub_frame[f"AOIFixation_P{participant}"].iloc[0])
    if re_fixation == False and len_of_fixation > 0 and len(fixations) == 0:
        re_fixation = True

    fixations.append(FixationDuration)
    prev_participant = participant
    prev_aoi_idx = aoi_idx
    prev_algorithm = algorithm

  0%|          | 0/241583 [00:00<?, ?it/s]

### Calculate the AOI Based Eyetracking Metrics

In [13]:
# Melt the Dataframe to be thinner so that we have Algorithm TokenIdx and Participant and Keys per Fixation / Refixation
df_token_melted = pd.melt(df_token, id_vars=["Algorithm", "TokenIdx"], var_name="KindOfFixation", value_name="FixationDurations")

# Classify Participant and the Kind and Number of Fixations / Refixations
df_token_melted["Participant"] = df_token_melted["KindOfFixation"].apply(lambda x: int(x.split("_")[1][1:]))
df_token_melted["KindOfFixation"] = df_token_melted["KindOfFixation"].apply(lambda x: x.split("_")[0])
df_token_melted["KindOfFixation"] = df_token_melted["KindOfFixation"].apply(lambda x: "Fixation" if x == "TokenFixation" else "ReFixation")
df_token_melted["NumberOfFixations"] = df_token_melted["FixationDurations"].apply(lambda x: len(x))

# Get the number of Participants for further calculations
number_of_participants = len(participants)

# Calculate the First Fixation Duration per Participant per Token
df_token_melted["FirstFixationDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "FirstFixationDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) > 0 else None)

# Calculate the Single Fixation Duration per Participant per Token
df_token_melted["SingleFixationDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "SingleFixationDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) == 1 else None)

# Calculate the Gaze Duration per Participant per Token
df_token_melted["GazeDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "GazeDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: sum(x) if len(x) > 0 else None)

# Calculate the Total Time per Participant per Token
df_token_melted_total_time = df_token_melted.groupby(["Participant", "Algorithm", "TokenIdx"])\
    .agg({"FixationDurations": lambda x: sum(x.values.sum())})
df_token_melted_total_time = df_token_melted_total_time.rename(columns={"FixationDurations": "TotalTime"})

# Merge the Dataframes
df_token_melted = pd.merge(df_token_melted, df_token_melted_total_time, on=["Participant", "Algorithm", "TokenIdx"], how="left")

# Cast the Dataframes to the right datatype
df_token_melted["FirstFixationDuration"] = df_token_melted["FirstFixationDuration"].astype(float)
df_token_melted["SingleFixationDuration"] = df_token_melted["SingleFixationDuration"].astype(float)
df_token_melted["GazeDuration"] = df_token_melted["GazeDuration"].astype(float)
df_token_melted["TotalTime"] = df_token_melted["TotalTime"].astype(float).replace(0, np.nan)

# Read in the Skilllevel
df_skill = pd.read_csv(f"../data/24/dataEvaluation/data/filteredData/filtered_data.csv")
df_skill = df_skill[["Participant", "SkillScore"]]
df_skill = df_skill.drop_duplicates()

# Merge the Dataframes to combine metrics with the Skilllevel
df_metrics_skill = pd.merge(df_token_melted, df_skill, on=["Participant"], how="left")

# Helper Methods for the Metrics (2. Define because of nearness of the code)
def get_no_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 0]
    return df_fixations


def get_single_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[~df_fixations["TokenIdx"].isin(df_refixations["TokenIdx"].values)]
    # remove every entry from df fixations on where Number Of Fixations is not 1
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 1]
    return df_fixations

def get_multiple_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[(df_fixations["TokenIdx"].isin(df_refixations["TokenIdx"].values)) & (df_fixations["NumberOfFixations"] >= 1)]
    return df_fixations

def get_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on where no Fixation is found
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] >= 1]
    return df_fixations



# dataframe for number of fixations per participant
number_of_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])["NumberOfFixations"].count()
number_of_fixation_per_algorithm = number_of_fixation_per_algorithm.reset_index()

# dataframe for number of tokens per algorithm
number_of_tokens_per_algorithm = df_metrics_skill.groupby(["Algorithm"])["TokenIdx"].max()
number_of_tokens_per_algorithm = number_of_tokens_per_algorithm.reset_index()

# dataframe for number of tokens with no fixation per algorithm per participant
df_no_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_no_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with only one fixation per algorithm per participant
df_single_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_single_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with more than one fixation per algorithm per participant
df_multiple_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_multiple_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with more or equal than one fixation per algorithm per participant
df_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_fixations)\
    .drop(["Algorithm"], axis=1)

# Reformat the dataframes
no_fixation_per_algorithm = df_no_fixation_per_algorithm[["TokenIdx"]]
no_fixation_per_algorithm = no_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

single_fixation_per_algorithm = df_single_fixation_per_algorithm[["TokenIdx"]]
single_fixation_per_algorithm = single_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

multiple_fixation_per_algorithm = df_multiple_fixation_per_algorithm[["TokenIdx"]]
multiple_fixation_per_algorithm = multiple_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

fixations_per_algorithm = df_fixation_per_algorithm[["TokenIdx"]]
fixations_per_algorithm = fixations_per_algorithm.reset_index().drop(["level_2"], axis=1)

# Helper Method for Probability Metrics
def group_len_divided_by_number(current_df, counting_df):
    algorithm = current_df["Algorithm"].iloc[0]
    number_of_tokens = counting_df[counting_df["Algorithm"] == algorithm]["TokenIdx"].iloc[0]
    value = len(current_df) / number_of_tokens
    return len(current_df) / (number_of_tokens + 1)

# Calculate the Metrics per Participant
# Probability of no fixation
df_no_fixation_probability = no_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_no_fixation_probability = df_no_fixation_probability.reset_index()

# Probability of single fixation
df_single_fixation_probability = single_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_single_fixation_probability = df_single_fixation_probability.reset_index()

# Probability of multiple fixation
df_multiple_fixation_probability = multiple_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_multiple_fixation_probability = df_multiple_fixation_probability.reset_index()

# Probability of fixation
df_fixation_probability = fixations_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_fixation_probability = df_fixation_probability.reset_index()

# Calculate the means for the metrics per algorithm
df_no_fixation_probability = df_no_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_single_fixation_probability = df_single_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_multiple_fixation_probability = df_multiple_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_fixation_probability = df_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)

# Raw Durations Metrics
# Duration of first fixation
df_first_fixation = df_metrics_skill[~df_metrics_skill["FirstFixationDuration"].isnull()]
df_first_fixation = df_first_fixation.groupby(["Participant"])["FirstFixationDuration"].mean()

# Duration of single fixation
df_single_fixation = df_metrics_skill[~df_metrics_skill["SingleFixationDuration"].isnull()]
df_single_fixation = df_single_fixation.groupby(["Participant"])["SingleFixationDuration"].mean()

# Duration of gaze duration
df_gaze_duration = df_metrics_skill[~df_metrics_skill["GazeDuration"].isnull()]
df_gaze_duration = df_gaze_duration.groupby(["Participant"])["GazeDuration"].mean()

# Total Time
df_total_time = df_metrics_skill[~df_metrics_skill["TotalTime"].isnull()]
df_total_time = df_total_time.groupby(["Participant"])["TotalTime"].mean()

# Put every metric dataframe together into one
df_combined = pd.DataFrame({"FirstFixationDuration": df_first_fixation.values,
                            "SingleFixationDuration": df_single_fixation.values,
                            "GazeDuration": df_gaze_duration.values,
                            "TotalTime": df_total_time.values,
                            "TokenNoFixationProbability": df_no_fixation_probability.values.reshape(37, ),
                            "TokenSingleFixationProbability": df_single_fixation_probability.values.reshape(37, ),
                            "TokenMultipleFixationProbability": df_multiple_fixation_probability.values.reshape(37, ),
                            "TokenFixationProbability": df_fixation_probability.values.reshape(37, ),
                            "Skill": df_metrics_skill.groupby(["Participant"])["SkillScore"].mean().values})
# get spearman correlation for metrics and skill level
df_combined.corrwith(df_metrics_skill["SkillScore"])

FirstFixationDuration               1.205543e-15
SingleFixationDuration              2.562218e-16
GazeDuration                       -9.878046e-16
TotalTime                           9.554281e-18
TokenNoFixationProbability          1.178154e-15
TokenSingleFixationProbability      1.179129e-17
TokenMultipleFixationProbability   -1.071257e-16
TokenFixationProbability           -9.268668e-16
Skill                              -5.638273e-16
dtype: float64

# Get the LOCs

In [14]:
# Get Bounding Boxes for Lines Of Code
snippets = df_fixation["Algorithm"].unique()
df_lines = pd.DataFrame(columns=["Algorithm", "Line", "BoundingBox"])
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}_ast.json"
    image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/")
    height, width = image.size
    width_offset = int(1920 * 0.5) - int(height / 2)
    height_offset = int(1080 * 0.5) - int(width / 2)
    aoi_clustered = []
    current_left = None
    current_top = None
    current_right = None
    current_bottom = None
    current_line = 0
    for letter in aoi_list:
        if letter["letter"] == '\n':
            if current_left is not None:
                aoi_clustered.append((current_line, current_left, current_top, current_right, current_bottom))
            current_left = None
            current_top = None
            current_right = None
            current_bottom = None
            current_line += 1
            continue
        if current_left is None:
            current_left = letter["BoundingBox"][0]
            current_top = letter["BoundingBox"][1]
            current_right = letter["BoundingBox"][2]
            current_bottom = letter["BoundingBox"][3]
        else:
            current_left = min(current_left, letter["BoundingBox"][0])
            current_top = min(current_top, letter["BoundingBox"][1])
            current_right = max(current_right, letter["BoundingBox"][2])
            current_bottom = max(current_bottom, letter["BoundingBox"][3])

    for token in aoi_clustered:
        df_lines.loc[len(df_lines)] = [snippet, token[0],
                                       (token[1] + width_offset,
                                        token[2] + height_offset,
                                        token[3] + width_offset,
                                        token[4] + height_offset)]
df_lines

  0%|          | 0/32 [00:00<?, ?it/s]

AOI not closed - None
AOI not closed - None
AOI not closed - None


,Algorithm,Line,BoundingBox
0,IsPrime,0,"(758, 467, 1142, 482)"
1,IsPrime,1,"(758, 486, 1102, 501)"
2,IsPrime,2,"(758, 505, 1039, 519)"
3,IsPrime,3,"(758, 526, 998, 539)"
4,IsPrime,4,"(758, 543, 870, 557)"
...,...,...,...
429,Rectangle,12,"(729, 601, 1001, 615)"
430,Rectangle,13,"(729, 619, 777, 633)"
431,Rectangle,15,"(729, 657, 945, 672)"
432,Rectangle,16,"(729, 676, 1089, 691)"


In [15]:
df_line_fixation_per_participant = pd.DataFrame([], columns=["Algorithm", "Participant", "FixationNumber", "FixationStart", "FixationEnd", "LineNumber"])
participants = df_fixation["Participant"].unique()
for snippet in tqdm(snippets):
    df_token_per_algo = df_lines[df_lines["Algorithm"] == snippet]

    for participant in participants:
        df_fixation_participant = df_fixation[(df_fixation["Algorithm"] == snippet) & (df_fixation["Participant"] == participant)]
        if len(df_fixation_participant) == 0:
            continue
        start_times = eval(df_fixation_participant["Fixation_startT"].values[0])
        end_times = eval(df_fixation_participant["Fixation_endT"].values[0])
        y_coordinates = eval(df_fixation_participant["Fixation_y"].values[0])
        y_range = eval(df_fixation_participant["Fixation_y_range"].values[0])
        idx_values = range(len(start_times))
        for (fix_idx, start, end, y, y_range) in zip(idx_values, start_times, end_times, y_coordinates, y_range):
            low_y = int(float(y) - math.ceil(float(y_range)))
            high_y = int(float(y) + math.ceil(float(y_range)))
            possible_coordinates = [y for y in range(low_y, high_y + 1)]

            found = False
            for idx, row in df_token_per_algo.iterrows():
                line_number = row["Line"]
                bounding_box = row["BoundingBox"]

                for possible_y in possible_coordinates:
                    if bounding_box[1] <= possible_y <= bounding_box[3]:
                        df_line_fixation_per_participant.loc[len(df_line_fixation_per_participant)] = [snippet, participant, fix_idx, start, end, line_number]
                        found = True
                        break
                if found:
                    break

df_line_fixation_per_participant

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,Participant,FixationNumber,FixationStart,FixationEnd,LineNumber
0,IsPrime,1,3,1136.027,1336.034,0
1,IsPrime,1,4,1356.034,1516.039,0
2,IsPrime,1,5,1536.038,1816.047,0
3,IsPrime,1,6,1840.047,2136.055,0
4,IsPrime,1,7,2156.055,2936.074,0
...,...,...,...,...,...,...
77836,Rectangle,71,26,8160.260,8316.263,7
77837,Rectangle,71,27,8332.263,8524.265,13
77838,Rectangle,71,29,8800.282,8952.286,3
77839,Rectangle,71,31,9428.297,9600.305,2


In [16]:
# Calculate the LOCs
snippets = df_fixation["Algorithm"].unique()
df_snippet_length = pd.DataFrame(columns=["Algorithm", "LOC"])
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}_ast.json"
    with open(aoi_token_generator) as f:
        aoi_list = json.load(f)
        data = aoi_list["source-code"]
        LOC = len(data)
        df_snippet_length.loc[len(df_snippet_length)] = [snippet, LOC]
df_snippet_length

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,LOC
0,IsPrime,8
1,SiebDesEratosthenes,20
2,IsAnagram,28
3,RemoveDoubleChar,19
4,BinToDecimal,9
5,PermuteString,23
6,Power,7
7,BinarySearch,15
8,ContainsSubstring,21
9,ReverseArray,7


In [17]:
# read in Behavioural and Skills data
df_behavioral = pd.read_csv('../data/24/dataEvaluation/data/filteredData/fixation_stats.csv', sep=";")
df_behavioral = df_behavioral[df_behavioral["IsOutlier"] == False]
df_behavioral = df_behavioral[["Participant", "Algorithm", "Duration", "SkillScore"]]
df_behavioral

,Participant,Algorithm,Duration,SkillScore
0,1,IsPrime,12.390280,0.331385
1,1,SiebDesEratosthenes,152.571914,0.331385
2,1,IsAnagram,109.615724,0.331385
3,1,RemoveDoubleChar,53.456276,0.331385
4,1,BinToDecimal,49.922091,0.331385
...,...,...,...,...
1066,71,GreatestCommonDivisor,30.757360,0.435651
1067,71,DumpSorting,113.368945,0.435651
1068,71,BinomialCoefficient,50.637861,0.435651
1069,71,IsAnagram,110.995754,0.435651


In [18]:
# merge dataframes together to access every possible combination of participant, algorithm and duration,skillscore
df_combined = pd.merge(df_line_fixation_per_participant, df_snippet_length, on=["Algorithm"])
df_combined = pd.merge(df_combined, df_behavioral, on=["Participant", "Algorithm"])

# transform the data to seconds
df_combined["FixationStart"] = df_combined["FixationStart"] / 1000.0
df_combined["FixationEnd"] = df_combined["FixationEnd"] / 1000.0

# Helper function to calculate the coverage of LOC after 'percentage' percent from the whole duration
def loc_coverage_after_time_percentage(df, percentage):
    end_duration = df["Duration"].iloc[0]
    loc = df["LOC"].iloc[0]
    max_duration = end_duration * percentage
    df_filtered = df[df["FixationEnd"] <= max_duration]
    unique_loc = df_filtered["LineNumber"].nunique()
    return unique_loc / loc

# calculate LOC coverage after a certain percentage of the duration
df_10 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.1))
df_20 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.2))
df_30 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.3))
df_40 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.4))
df_50 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.5))
df_60 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.6))
df_70 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.7))
df_80 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.8))
df_90 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.9))
df_100 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 1.))

df_10 = df_10.reset_index()
df_20 = df_20.reset_index()
df_30 = df_30.reset_index()
df_40 = df_40.reset_index()
df_50 = df_50.reset_index()
df_60 = df_60.reset_index()
df_70 = df_70.reset_index()
df_80 = df_80.reset_index()
df_90 = df_90.reset_index()
df_100 = df_100.reset_index()


df_10 = df_10.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_20 = df_20.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_30 = df_30.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_40 = df_40.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_50 = df_50.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_60 = df_60.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_70 = df_70.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_80 = df_80.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_90 = df_90.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_100 = df_100.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )

# extract the raw skillscore per participant
df_skill = df_behavioral[["Participant", "SkillScore"]]
df_skill = df_skill.drop_duplicates()

# create a dataframe with all the LOC coverage and Participant
df_code_coverage = pd.DataFrame({"Participant": participants,
                                 "10%" : df_10,
                                 "20%" : df_20,
                                 "30%" : df_30,
                                 "40%" : df_40,
                                 "50%" : df_50,
                                 "60%" : df_60,
                                 "70%" : df_70,
                                 "80%" : df_80,
                                 "90%" : df_90,
                                 "100%" : df_100})

df_code_coverage.set_index("Participant", inplace=True, drop=True)

# Merge LOC coverage with skillscore with correlations
df_code_coverage = pd.merge(df_code_coverage, df_skill, on=["Participant"])
df_code_coverage.set_index("Participant", inplace=True, drop=True)
df_code_coverage

,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%,SkillScore
Participant,,,,,,,,,,,
1,0.102645,0.245391,0.350040,0.434370,0.494765,0.548473,0.587523,0.618069,0.672051,0.702769,0.331385
2,0.145467,0.251232,0.352979,0.420231,0.465964,0.500237,0.558354,0.597333,0.648105,0.681686,0.379187
3,0.290797,0.415433,0.518259,0.638263,0.721339,0.768317,0.810514,0.820689,0.828467,0.849472,0.311264
4,0.081401,0.245859,0.331361,0.427805,0.500664,0.554532,0.594059,0.635949,0.664308,0.712249,0.424727
5,0.201621,0.298898,0.392059,0.471176,0.540164,0.636644,0.678595,0.699955,0.729923,0.774195,0.313031
6,0.120993,0.252214,0.382175,0.478461,0.571992,0.632226,0.689519,0.747845,0.798724,0.816875,0.315932
7,0.121032,0.267153,0.390450,0.460819,0.497120,0.535291,0.568928,0.605776,0.635783,0.655603,0.420873
10,0.076049,0.154950,0.243787,0.325329,0.406281,0.448024,0.502428,0.555074,0.591713,0.621052,0.350392
11,0.143196,0.332465,0.460923,0.524091,0.603788,0.648332,0.706987,0.735700,0.764694,0.832834,0.178206


In [19]:
# describe the LOC coverage
df_code_coverage.describe()

,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%,SkillScore
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
mean,0.109584,0.218750,0.310104,0.389436,0.461383,0.512549,0.562781,0.604019,0.643851,0.691354,0.325230
std,0.093647,0.114325,0.126105,0.128450,0.123490,0.119174,0.114791,0.110564,0.106907,0.102052,0.102510
min,0.000000,0.044056,0.118774,0.193105,0.270581,0.330115,0.378934,0.394535,0.394535,0.410324,0.142520
25%,0.038878,0.123741,0.219552,0.287566,0.365590,0.421855,0.470120,0.530543,0.577918,0.641630,0.259896
50%,0.096695,0.236237,0.320445,0.389047,0.465964,0.507033,0.561610,0.605776,0.648105,0.687911,0.315932
75%,0.144094,0.260851,0.354411,0.460819,0.516911,0.571973,0.636023,0.670271,0.713262,0.774195,0.379187
max,0.418425,0.552526,0.687982,0.765678,0.797907,0.808313,0.824526,0.846036,0.867382,0.888266,0.656726


In [23]:
# spearman correlation for LOC coverage and skillscore
LOC_corr = df_code_coverage.corrwith(df_code_coverage["SkillScore"], method = "spearman")
LOC_corr

10%          -0.074680
20%          -0.084400
30%          -0.114272
40%          -0.106686
50%          -0.168326
60%          -0.215742
70%          -0.260550
80%          -0.266240
90%          -0.288051
100%         -0.342105
SkillScore    1.000000
dtype: float64

In [21]:
# Save the Dataframe to a CSV
LOC_corr.to_csv("./original_24_code_coverage.csv", index=True)

In [22]:
from matplotlib.patches import Circle
import matplotlib.pyplot as plt
import os


# check if folder exists and create it if not
if not os.path.exists("../data/24/dataEvaluation/data/RQ1/plots/scanpaths"):
    os.makedirs("../data/24/dataEvaluation/data/RQ1/plots/scanpaths")

if not os.path.exists("../data/24/dataEvaluation/data/RQ1/plots/vertical_scanpaths"):
    os.makedirs("../data/24/dataEvaluation/data/RQ1/plots/vertical_scanpaths")

def clamp(low, value, high):
    return max(low, min(value, high))

snippets = df_fixation["Algorithm"].unique()
# iterate over all algorithms
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}_ast.json"
    image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/")
    width, height = image.size
    x_low = int(1920 * 0.5) - int(width / 2)
    y_low = int(1080 * 0.5) - int(height / 2)
    x_high = int(1920 * 0.5) + int(width / 2)
    y_high = int(1080 * 0.5) + int(height / 2)
    df_algo = df_fixation[df_fixation["Algorithm"] == snippet]
    df_algo = df_algo.sort_values(by=["SkillScore"])
    participants = df_algo["Participant"].unique()
    # iterate over all participants
    for participant in participants:
        if len(df_algo[df_algo["Participant"] == participant]) != 1:
            display(df_algo[df_algo["Participant"] == participant])
            raise ValueError("Participant has more than not exactly one entry")
        df_algo_part = df_algo[df_algo["Participant"] == participant].reset_index().iloc[0]
        skillscore = df_algo_part["SkillScore"]
        current_image = image.copy()
        fixation_start_array = np.array(eval(df_algo_part["Fixation_startT"]))
        fixation_end_array = np.array(eval(df_algo_part["Fixation_endT"]))
        fixation_x_coordinates = np.array(eval(df_algo_part["Fixation_x"]))
        fixation_y_coordinates = np.array(eval(df_algo_part["Fixation_y"]))
        fixations = np.stack((fixation_start_array, fixation_end_array, fixation_x_coordinates, fixation_y_coordinates), axis=1)

        cm = plt.cm.get_cmap('inferno')
        cm = plt.cm.ScalarMappable(cmap=cm, norm=plt.Normalize(vmin=0, vmax=len(fixations)))
        patches = []

        # total scanpath
        for idx, (start, end, x, y) in enumerate(fixations):
            if x_low <= x <= x_high and y_low <= y <= y_high:
                x = int(x)
                y = int(y)
                x = x - x_low
                y = y - y_low
                color = cm.to_rgba(idx)
                patches.append(Circle((x, y), radius=10, color=color, alpha=0.5))

        fig, ax = plt.subplots(1)
        ax.imshow(current_image)
        for p in patches:
            ax.add_patch(p)

        for idx in range(len(fixations) - 1):
            start_x = fixations[idx, 2] - x_low
            start_y = fixations[idx, 3] - y_low
            end_x = fixations[idx+1, 2] - x_low
            end_y = fixations[idx+1, 3] - y_low
            if 0 <= start_x <= width and 0 <= start_y <= height and 0 <= end_x <= width and 0 <= end_y <= height:
                color = cm.to_rgba(idx)
                ax.plot([start_x, end_x], [start_y, end_y], color=color, alpha=0.3)

        plt.xlim(0, width)
        plt.ylim(height, 0)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f"../data/24/dataEvaluation/data/RQ1/plots/scanpaths/scanpath_{snippet}_{skillscore}_{participant}.pdf")
        plt.close()

        # vertical scanpath
        vertical_points_and_time = []
        for idx, (start, end, x, y) in enumerate(fixations):
            if x_low <= x <= x_high and y_low <= y <= y_high:
                y = int(y) - y_low
                duration = (end - start) / 1000
                vertical_points_and_time.append((y, duration))

        pixel_per_second = 3
        current_x = 0
        if len(vertical_points_and_time) == 0:
            continue
        line_points = [(current_x, vertical_points_and_time[0][0])]
        current_x += vertical_points_and_time[0][1] * pixel_per_second
        line_points.append((current_x, vertical_points_and_time[0][0]))
        for idx, (y, duration) in enumerate(vertical_points_and_time[1:]):
            line_points.append((current_x, y))
            current_x += duration * pixel_per_second
            line_points.append((current_x, y))

        max_x = max([x for x, y in line_points])
        line_points = [(x - max_x, y) for x, y in line_points]
        cm = plt.cm.get_cmap('inferno')
        cm = plt.cm.ScalarMappable(cmap=cm, norm=plt.Normalize(vmin=0, vmax=len(line_points)))
        fig, ax = plt.subplots(1)
        ax.imshow(current_image)
        for idx in range(len(line_points) - 1):
            start_x = line_points[idx][0]
            start_y = line_points[idx][1]
            end_x = line_points[idx+1][0]
            end_y = line_points[idx+1][1]
            color = cm.to_rgba(idx)
            ax.plot([start_x, end_x], [start_y, end_y], color=color)
        plt.xlim(-max_x, width)
        plt.ylim(height, 0)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f"../data/24/dataEvaluation/data/RQ1/plots/vertical_scanpaths/scanpath_vertical_{snippet}_{skillscore}_{participant}.pdf", dpi=200)
        plt.close()

  0%|          | 0/32 [00:00<?, ?it/s]

C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykerne

AOI not closed - None


C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykerne

AOI not closed - None


C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykerne

AOI not closed - None


C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_55636\1493370360.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykerne